In [1]:
import pandas as pd
import numpy as np
import itertools
from datetime import datetime,timedelta,date
from collections import defaultdict

#import colorlover as cl
#import random
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import plotly
#import plotly.graph_objs as go
#from plotly import tools

#import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
#df = pd.read_csv('../features/clear_data/rm_features.csv',';',infer_datetime_format=True,index_col=['time','host','consumer_group'],parse_dates=['time'])
#df = pd.read_csv('../features/clear_data/rm_features.csv',';',infer_datetime_format=True,index_col=['time'],parse_dates=['time'])
df = pd.read_csv('../features/clear_data/rm_features.csv',';',infer_datetime_format=True,parse_dates=['time'])

In [3]:
host_cons=[(host,group) for host,group in df.drop_duplicates(['host','consumer_group'])[['host','consumer_group']].values]


In [4]:
X_train=pd.DataFrame()
X_pred=pd.DataFrame()
for (host,consumer_group) in host_cons:
    sample=df[(df.host==host)&(df.consumer_group==consumer_group)]
    X_train=pd.concat([X_train,sample.head(int(len(sample)*0.97))],sort=False)
    X_pred=pd.concat([X_pred,sample[~sample.index.isin(X_train.index)]],sort=False)
#X_train=X_train.set_index(['time','host','consumer_group'])
#X_pred=X_pred.set_index(['time','host','consumer_group'])

In [ ]:
# Разбить временной ряд на периоды с некоторым окном и свернуть каждый период с помощью PCA

#ts_period=1 # количество дней 
#n_length=24 # количество часов


#def window_stack(a, window=ts_period*n_length):
    # разбиваем данные на окна длиной ts_period
#    X=np.array([a[i:window+i] for i in range(int(len(a) - window))])
#    return X


#from sklearn.decomposition import PCA
#model_PCA = PCA(n_components=1)

#for (host,consumer_group) in host_cons:
#    samples=window_stack(df[(df.host==host)&(df.consumer_group==consumer_group)].set_index(['host','consumer_group']).values)
    #model_PCA.fit_transform(df)
#    for sample in samples:
        

In [37]:
outliers_fraction = 0.01
def predict(model,X_train,X_pred):
    model_predict=pd.DataFrame()
    for (host,consumer_group) in host_cons:
        sample_train=X_train[(X_train.host==host)&(X_train.consumer_group==consumer_group)].set_index(['time','host','consumer_group'])
        sample_pred=X_pred[(X_pred.host==host)&(X_pred.consumer_group==consumer_group)].set_index(['time','host','consumer_group'])
        model_predict=pd.concat([model_predict,pd.DataFrame(model.fit(sample_train).predict(sample_pred),columns=['is_anomaly'])],sort=False)
    return model_predict.reset_index(drop=True)
        


In [ ]:
#outliers_fraction = 0.005
#def predict(model,X_train,X_pred):
#    return pd.DataFrame(model.fit(X_train).predict(X_pred),columns=['is_anomaly'])
        

## ML models

In [38]:
from sklearn.ensemble import IsolationForest

rns = np.random.RandomState(42)
model_IF = IsolationForest(random_state=rns,contamination=outliers_fraction,n_estimators =1000)

anomaly_IF = predict(model_IF,X_train,X_pred)



In [ ]:
#from sklearn.neighbors import LocalOutlierFactor

#model_LOF = LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction)
#anomaly_LOF = predict(model_LOF,X_train,X_pred)

In [39]:
from sklearn import svm

model_SVM = svm.OneClassSVM(random_state=rns, nu=outliers_fraction, kernel="rbf", gamma=0.01)
anomaly_SVM = predict(model_SVM,X_train,X_pred)

In [40]:
from sklearn.covariance import EllipticEnvelope

model_EE = EllipticEnvelope(contamination=outliers_fraction)
anomaly_EE = predict(model_EE,X_train,X_pred)

C:\Users\JustMe\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:622: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


In [ ]:
#from sklearn.decomposition import PCA

#model_PCA = PCA(n_components=15, svd_solver='full')
#model_PCA.fit_transform(df)

#w_components_ = np.ones([model_PCA.n_components_, ])
#selected_components_ = model_PCA.components_[-1 * model_PCA.n_selected_components_:, :]
#selected_w_components_ = w_components_[-1 * model_PCA.n_selected_components_:]

In [41]:
tmp=anomaly_IF[anomaly_IF.is_anomaly==-1].join(anomaly_SVM[anomaly_SVM.is_anomaly==-1],rsuffix ='_',how='outer').join(anomaly_EE[anomaly_EE.is_anomaly==-1],rsuffix ='__',how='outer')
tmp=tmp[tmp.sum(axis=1)<-1]
df_predict=tmp.join(X_pred.reset_index()[['time','host','consumer_group']])
df_predict

,is_anomaly,is_anomaly_,is_anomaly__,time,host,consumer_group
28,NaN,-1.0,-1.0,2018-12-10 23:00:00,1,adw_unload_group
33,-1.0,-1.0,NaN,2018-12-09 23:00:00,1,fp_dm_prior
63,-1.0,-1.0,NaN,2018-12-10 00:00:00,1,hdp_unload_group
128,-1.0,-1.0,-1.0,2018-12-10 08:00:00,2,adw_unload_group
129,-1.0,-1.0,NaN,2018-12-10 09:00:00,2,adw_unload_group
143,NaN,-1.0,-1.0,2018-12-10 23:00:00,2,adw_unload_group
157,-1.0,-1.0,-1.0,2018-12-10 08:00:00,2,fp_dm_prior
178,-1.0,-1.0,NaN,2018-12-10 00:00:00,2,hdp_unload_group
234,-1.0,-1.0,-1.0,2018-12-09 23:00:00,3,adw_unload_group
235,-1.0,-1.0,NaN,2018-12-10 00:00:00,3,adw_unload_group


## Autoencoders

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
from keras.callbacks import TensorBoard


dim=X_train.set_index(['time','host','consumer_group']).shape[1]

X_input = Input(shape=(dim,))
encoded = Dense(64, activation='relu', activity_regularizer=regularizers.l1(10e-5))(X_input)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(dim, activation='sigmoid')(decoded)

autoencoder = Model(X_input, decoded)

def autoencoder_fit_predict(model,X_train,X_pred):
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, X_train,
                    epochs=20,
                    batch_size=64,
                    shuffle=True,
                    validation_data=(X_pred, X_pred))
    return model.predict(X_pred)

model_predict=pd.DataFrame()
for (host,consumer_group) in host_cons:
    sample_train=X_train[(X_train.host==host)&(X_train.consumer_group==consumer_group)].set_index(['time','host','consumer_group'])
    sample_pred=X_pred[(X_pred.host==host)&(X_pred.consumer_group==consumer_group)].set_index(['time','host','consumer_group'])
    
    sample_decoded=pd.DataFrame(autoencoder_fit_predict(autoencoder,sample_train,sample_pred),columns=sample_pred.columns)
    autoencoder_predict=pd.concat([sample_pred.reset_index()[['time','host','consumer_group']],sample_decoded],axis=1,sort=False)

    model_predict=pd.concat([model_predict,autoencoder_predict],sort=False)


In [ ]:
model_predict

In [ ]:
mse = np.mean(np.power(X_pred.drop(columns=['time','host','consumer_group']).values - model_predict.drop(columns=['time','host','consumer_group']).values, 2), axis=1)


In [ ]:
mse

In [ ]:
plt.plot(mse.loc[(3,'adw_unload_group')][['time']], mse.loc[(1,'adw_unload_group')][['mse']], '*')
plt.xticks(rotation='vertical')